## SD2 Img-to-Img and Inpainting - Gradio for Share

In [1]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/ubuntu/.huggingface/token
Login successful


In [2]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A10G, 23028 MiB, 22592 MiB


In [2]:
!pip install -q --upgrade diffusers==0.10.0 transformers ftfy scipy
!pip install -qq "ipywidgets>=7,<8"

In [3]:
!pip install --upgrade triton

  Using cached triton-1.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 0.4.2
    Uninstalling triton-0.4.2:
      Successfully uninstalled triton-0.4.2


In [4]:
!pip install -q altair

In [5]:
!pip install -q accelerate==0.12.0 bitsandbytes gradio natsort

In [8]:
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

import requests
from PIL import Image
from io import BytesIO
import gradio as gr

from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionInpaintPipeline


In [7]:
from diffusers import DDIMScheduler, PNDMScheduler, LMSDiscreteScheduler, DDPMScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler


In [9]:
pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2",
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to('cuda')


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
).to("cuda")

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [11]:
def scheduler_i2i(option):
    if option =='PNDM':
        pipe_i2i.scheduler=PNDMScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'DDIM':
        pipe_i2i.scheduler=DDIMScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'LMSDiscrete':
        pipe_i2i.scheduler=LMSDiscreteScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'DDPM':
        pipe_i2i.scheduler=DDPMScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'EulerDiscrete':
        pipe_i2i.scheduler=EulerDiscreteScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'EulerAncestral':
        pipe_i2i.scheduler=EulerAncestralDiscreteScheduler.from_config(pipe_i2i.scheduler.config)
    elif option == 'DPMSolverMultistep':
        pipe_i2i.scheduler=DPMSolverMultistepScheduler.from_config(pipe_i2i.scheduler.config)  
    return pipe_i2i.scheduler


In [12]:
def scheduler_inpaint(option):
    if option =='PNDM':
        pipe_inpaint.scheduler=PNDMScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'DDIM':
        pipe_inpaint.scheduler=DDIMScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'LMSDiscrete':
        pipe_inpaint.scheduler=LMSDiscreteScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'DDPM':
        pipe_inpaint.scheduler=DDPMScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'EulerDiscrete':
        pipe_inpaint.scheduler=EulerDiscreteScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'EulerAncestral':
        pipe_inpaint.scheduler=EulerAncestralDiscreteScheduler.from_config(pipe_inpaint.scheduler.config)
    elif option == 'DPMSolverMultistep':
        pipe_inpaint.scheduler=DPMSolverMultistepScheduler.from_config(pipe_inpaint.scheduler.config)  
    return pipe_inpaint.scheduler

In [13]:
def re_size(img):
    """Resize (scale down) input image size, if max length > 768. Width and height ratio kept same as original."""
    
    bigger = max(img.size[0], img.size[1])
    if bigger > 768:
        ratio = 768 / bigger
        new_width = int(img.size[0] * ratio)
        new_height = int(img.size[1] * ratio)
        img = img.resize((new_width, new_height))
    else:
        img = img
    return img  

In [14]:
generator = torch.Generator('cuda')

In [15]:
def img_img(init_img, prompt, n_prompt, option, strength, guidance_scale, num_images, manual_seed):    
    generator = torch.Generator('cuda')
    scheduler_i2i(option)    
    init_img=init_img.convert("RGB")
    init_img = re_size(init_img)
    
    if manual_seed != "":
        manual_seed = int(manual_seed)
        
        with autocast("cuda"):
            image = pipe_i2i(image=init_img, 
                         prompt=prompt, 
                         negative_prompt=n_prompt, 
                         strength=strength, 
                         guidance_scale=guidance_scale, 
                         generator=generator.manual_seed(manual_seed)
                        ).images[0]
            images = [image, image]
            seeds = manual_seed
         
    if manual_seed == "":
        seeds = []
        for i in range(num_images):
            seed = generator.seed()
            seeds.append(seed)
        
        images=[]
        with autocast("cuda"):
            for seed in seeds:
                image = pipe_i2i(image=init_img, 
                             prompt=prompt, 
                             negative_prompt=n_prompt, 
                             strength=strength, 
                             guidance_scale=guidance_scale, 
                             generator=generator.manual_seed(seed)
                            ).images[0]
                images.append(image)
                
    return images, str(seeds)

In [16]:
def inpaint(dict, prompt, n_prompt, option, inference_steps, guidance_scale, num_images):  
    scheduler_inpaint(option)
    image =  dict['image'].convert("RGB")
    image = re_size(image)
    mask_image = dict['mask'].convert("RGB")
    mask_image = re_size(mask_image)
    
    seeds = []
    for i in range(num_images):
        seed = generator.seed()
        seeds.append(seed)

    images=[]
    with autocast("cuda"):
        for seed in seeds:   
            output = pipe_inpaint(prompt=prompt, 
                          negative_prompt=n_prompt, 
                          image=image, 
                          mask_image=mask_image, 
                          width = image.size[0],
                          height = image.size[1],
                          num_inference_steps=inference_steps,
                          guidance_scale=guidance_scale,
                          generator=generator.manual_seed(seed)  
                          ).images[0]
            images.append(output)
    return images

In [ ]:
with gr.Blocks() as demo:
    with gr.Tab(label="Image-to-Image"):
        with gr.Row():
            markdown = gr.Markdown("Upload an initial sketch or image to be used as a reference.")
        with gr.Row():
            with gr.Column(scale=1):
                init_img = gr.Image(source = 'upload', type = 'pil')
                prompt = gr.Textbox(label = 'Prompt', placeholder="Describe the image you would like to generate")
                n_prompt = gr.Textbox(label = 'Negative Prompt', placeholder="Unwanted in the generated image")
                strength = gr.Slider(minimum=0, maximum=1, value=0.75, step=0.05, label='Variation from Initial Image', interactive=True)
                guidance_scale = gr.Slider(minimum=0, maximum=20, value=8.5, step=0.5, label='Prompt Guidance Scale (Typically 7.5 - 12.5)', interactive=True)
                num_images = gr.Slider(minimum=1, maximum=4, value=2, step=1, label='Number of Images per Run', interactive=True)
                manual_seed = gr.Textbox(label = 'Choose One Seed Number, Otherwise Automatically Generated')
            with gr.Column(scale=1):
                output1 = gr.Gallery()
                with gr.Row():
                    btn1 = gr.Button(value="Generate Image")
                    btn2 = gr.Button(value="TBD")
                output2 = gr.Textbox(label = 'Seeds Used')
        with gr.Row():      
            option = gr.Radio(choices=['DDIM', 'PNDM', 'LMSDiscret', 'DDPM', 'EulerDiscrete', 'EulerAncestral', 
                                       'DPMSolverMultistep'], value='DDIM', label='Choose Inference Schedule Method', interactive=True)
        option.change(fn=scheduler_i2i, inputs=option)
        btn1.click(fn=img_img, inputs=[init_img, prompt, n_prompt, option, strength, guidance_scale, num_images, manual_seed], 
                   outputs=[output1, output2])
            
    with gr.Tab(label="Inpaint"):
        with gr.Row():
            markdown = gr.Markdown("Upload an image and draw mask where you want a change.")
        with gr.Row():
            with gr.Column(scale=1):
                dict=gr.Image(source = 'upload', tool = 'sketch', type = 'pil')
                prompt = gr.Textbox(label = 'Prompt', placeholder="Describe what you want to replace the mask with")
                n_prompt = gr.Textbox(label = 'Negative Prompt', placeholder="Unwanted in the generated image") 
                num_images = gr.Slider(minimum=1, maximum=4, value=2, step=1, label='Number of Samples per Run', interactive=True)
                guidance_scale = gr.Slider(minimum=0, maximum=20, value=8.5, step=0.5, 
                                            label='Guidance Scale (Typically between 7.5 and 12.5)', interactive=True)                     
                inference_steps = gr.Slider(minimum=10, maximum=400, value=100, step=10, 
                                            label='Number of Steps for Inference', interactive=True)                        
            with gr.Column(scale=1):
                output = gr.Gallery()
                with gr.Row():
                    btn1 = gr.Button(value="Modify Image")
                    btn2 = gr.Button(value="TBD")
        with gr.Row():      
            option = gr.Radio(choices=['DDIM', 'PNDM', 'LMSDiscret', 'DDPM', 'EulerDiscrete', 'EulerAncestral', 
                                       'DPMSolverMultistep'], value='PNDM', label='Choose Inference Schedule Method', interactive=True)
        option.change(fn=scheduler_inpaint, inputs=option)
        btn1.click(fn=inpaint, inputs=[dict, prompt, n_prompt, option, inference_steps, guidance_scale, num_images], outputs=output)
        
demo.launch(debug=True, share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a2d9c6d89f0c4bc8.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/gradio/routes.py", line 297, in run_predict
    output = await app.blocks.process_api(
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/gradio/blocks.py", line 1007, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/gradio/blocks.py", line 848, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/conda/envs/pytorch/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/tmp/ipykernel_3

  0%|          | 0/240 [00:00<?, ?it/s]